In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

options = ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--disable-javascript")

driver = Chrome(options=options)

driver.get("https://patient.info/forums/discuss/browse/menopause-1411?page=97#group-discussions")

acceptBtn = driver.find_element(By.CSS_SELECTOR, ".css-11i4sx8")
acceptBtn.click()

pageCount = 1

# Open a CSV file for writing
# DO NOT PUT patient_posts.csv, this will rewrite the whole file.
with open('./temp_content.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write header
    csv_writer.writerow(["Author", "Datetime", "Title", "Content"])

    while True:
        postList = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".post__title"))
        )

        for i in range(len(postList)):
         
            # Check if postList has enough elements
            if len(postList) > i:
                post = postList[i]

                # Scroll to the post element before clicking
                driver.execute_script("arguments[0].scrollIntoView();", post)
                time.sleep(1)

                anchor = post.find_element(By.TAG_NAME, 'a')

                driver.execute_script("window.open(arguments[0].href,'_blank');", anchor)
                driver.switch_to.window(driver.window_handles[1])

                try:
                    # Wait for the post content to load
                    postArticle = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.TAG_NAME, "article"))
                    )

                    postContent = postArticle.find_element(By.CSS_SELECTOR, "#post_content")
        
                    postTitle = postArticle.find_element(By.CSS_SELECTOR, ".post__title")

                    postAuthor = postArticle.find_element(By.CSS_SELECTOR, ".author__name")

                    postTime = postArticle.find_element(By.TAG_NAME, "time").get_attribute("datetime")

                    excluded_elements = postContent.find_element(By.CLASS_NAME, 'post__stats')

                    # Clean the content
                    cleaned_content = postContent.text.replace(excluded_elements.text, '').replace('\n', ' ').strip()

                    # Print the content
                    print(f"Page {pageCount},Post {i}")
                    print(postTitle.text)

                    # Write the post content to the CSV file
                    csv_writer.writerow([postAuthor.text, postTime, postTitle.text, cleaned_content])

                finally:
                    # Navigate back to the previous page
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

        # Try to find the "Next" button
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.LINK_TEXT, 'Next'))  # You can change this selector based on the actual button text or class
            )
            next_button.click()  # Click the "Next" button to go to the next page
            time.sleep(2)  # Wait for the next page to load
            pageCount += 1
        except Exception as e:
            print("No more pages to navigate.")
            break  # Break the loop if no more pages are available

driver.quit()


KeyboardInterrupt: 

In [1]:
import pandas as pd

posts = pd.read_csv('patient_posts.csv')
posts['Datetime'] = pd.to_datetime(posts['Datetime'])
posts_sorted = posts.sort_values(by='Datetime', ascending=False).reset_index(drop=True)


posts_sorted

,Author,Datetime,Title,Content
0,vivian72668,2024-03-02 16:20:00+00:00,"PRE-MENOPAUSE: Ladies, who were you before MEN...","Hello There, Lovely Ladies Who were you, befor..."
1,vivian72668,2024-02-28 19:14:00+00:00,POST-MENOPAUSAL Nervous Bladder: Anyone Else?,"Hey Ladies... For the past week and a half, my..."
2,omachi26709,2024-02-28 10:46:00+00:00,Spotting for 6 weeks now,"Hello sweet ladies, i have been on peri since ..."
3,kate78592,2024-02-26 08:19:00+00:00,Am I too young to have perimenopausal symptoms,since i was about 38 ive had brown discharge a...
4,lamanp,2024-02-21 21:19:00+00:00,I’m dead with all this 😭,"Hello, all lady , I'm sitting here with tears ..."
...,...,...,...,...
3613,Lesley998,2012-08-14 18:55:00+00:00,Perimenopause and SEVERE joint pain - any one ...,Hi I am 50 and have been in perimenopause for ...
3614,Guest,2010-11-04 15:51:00+00:00,Livial/Tibolone Beware,I am 53yrs old and was precribed Tibolone in J...
3615,Guest,2009-03-04 00:04:00+00:00,Livial I love you!,:D I have been taking Livial for 4 months now ...
3616,Guest,2006-07-28 17:23:00+00:00,"Hi, I'm 46, and I think I may be having some M...","Hi, I'm 46, and I think I may be having some M..."
